In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import linprog, minimize_scalar
from mpl_toolkits import mplot3d

from functions import *

In [2]:
a = 0.01
type(a)

float

# Model

In matrix form for $Z = (z_1, ..., z_n) \in \mathbb{R}^{d\times n}$ this reads
\begin{equation*}
    Y = Q^* X (P^*)^\top + \sigma Z
\end{equation*}
where $P^*_{ij} = \mathbb{1}_{\pi^*(i) = j}$. To see that this corresponds to (1), note that $$
\left(X(P^*)^\top\right)_{ij}= \sum_{k=1}^n X_{ik} P^*_{jk} = \sum_{k=1}^n X_{ik} \mathbb{1}_{\pi^*(j) = k} = X_{i, \pi^*(j)} \quad \implies \quad \left(X(P^*)^\top\right)_{\bullet j} = x_{\pi^*(j)}
$$

# Optimisation algorithms

### Frank Wolfe

We need to use Frank-Wolfe for solving
$$
D^* = \underset{D \in \mathcal{D}_n}{\mathrm{argmin}} \, \Vert AD - DB \Vert_F^2
$$
where $A = X^TX$, $B = Y^\top Y$, and
$$
\mathcal{D}_n = \left\{ D \in \mathbb{R}^{n\times n} \, : \, \sum_{i = 1}^n D_{ij} = 1 \text{ and } \sum_{j = 1}^n D_{ij} = 1 \right\}.
$$

For this, define
$$
    f: \mathbb{R}^{n^2} \to \mathbb{R}, \, D = (d_{ij})_{i, j = 1}^n \, \mapsto \, \Vert AD - DB \Vert_F^2
$$
and realise that $f$ is differentiable as well as convex. Furthermore, the set $\mathcal{D}_n$ is convex and compact. We will generate a sequence $D_i$ converging to $D^*$. 

In [2]:
def frank_wolfe(D_init, X, Y, step_max, stepsize, A_eq, b_eq, tol=0.1, verbose=0):
    if verbose>0: print("Starting Frank-Wolfe with "+str(step_max)+ " steps...")
    D = D_init.copy()
    if step_max == 0: return D
    n = len(D)
    if verbose>0: print("Value of f before iterations = "+str(f(D, X, Y)))
    step = 1
    while True:
        if verbose>1: print("FW step no "+str(step))
        G = (grad_f(D, X, Y)).flatten()
        S_hat = perm_projection(G, A_eq, b_eq)
        Direction = S_hat - D
        # Line search
        gamma = minimize_scalar(
            lambda gamma:f(D + gamma*Direction), 
            bounds=(0,1), 
            method='bounded').x
        
        D = D + gamma*Direction
#         D = D + stepsize*Direction
#         D = D + (2/(step + 1))*Direction
        if step == step_max:
                if verbose>0: print("Reached max number of iterations.")
                break
        step += 1
    if verbose>0: 
        print("Value of f at convergence = "+str(f(D, X, Y)))
        print("Finished Frank-Wolfe...")
    return D

In [3]:
def pingpong(Pinit, Pstar, Qstar, X, Y, A_eq, b_eq, plot=True, verbose=0):
    loss_errs = []
    orth_errs = []
    perm_errs = []
    my_xticks = []

    pindex = 0
    qindex = 0
    P_turn = False
    n = len(X[0])
    d = len(X)
    lastP = np.zeros((n, n))
    lastQ = np.zeros((d, d))
    P = Pinit.copy()
    Q = np.ones((d, d))
    
    if verbose>0: print("Starting PingPong...")

    while np.linalg.norm(lastP-P) > 0 and np.linalg.norm(lastQ-Q) > 0:
        if P_turn:
            if verbose>1: print(str(is_permutation(P)) + str(np.linalg.norm(lastP-P)))
            pindex += 1
            lastP = P
            P = perm_from_ortho(Q, X, Y, A_eq, b_eq)
            P_turn = False
        else:
            qindex += 1
            lastQ = Q
            Q = ortho_from_perm(P, X, Y)
            P_turn = True
        loss_errs.append(empirical_loss(X, Y, P, Q)**2)
        perm_errs.append(L2perm_squared(P, Pstar, X))
        orth_errs.append(L2ortho_squared(Q, Qstar))
        my_xticks.append("(P"+str(pindex)+", Q"+str(qindex)+")")
    if verbose>0: print("Finished PingPong. Loss at convergence equals: " + str(empirical_loss(X, Y, P, Q)))
    if verbose>0: print("Loss at optimum (P*, Q*) equals: " + str(empirical_loss(X, Y, Pstar, Qstar)))

    if plot:
        xs = range(len(ys))
        plt.plot(xs, loss_errs, label="||QX - YX||^2")
        plt.plot(xs, orth_errs, label="||Q - Q*||^2")
        plt.plot(xs, perm_errs, label="||XP.T - XP*.T||^2")
        plt.xticks(xs, my_xticks)
        plt.legend()
        plt.show()
    return overlap(P, Pstar)


# Experiments

Here, we try the above functions for different values of $n$, $d$ and $\sigma$. 

In [4]:
def experiment(optim_function, 
               n, d, sigma, 
               P0_steps, 
               stepsize,
               tol=0.1,
               verbose=0, 
               pingpong_plot=False,
               init_plot=False,
               seed_XYZ = 1234
              ):
    # Setup
    A_eq, b_eq = equality_constraints(n)
    X, Y, Pstar, Qstar = initialise_XYPstarQstar(n, d, sigma, 
                                                 init_plot, 
                                                 seed_XYZ)
    # Find P0
    D_init = (1/n)*np.ones((n, n))
    P0 = optim_function(D_init,
                        X, Y, 
                        P0_steps, 
                        stepsize, 
                        A_eq, b_eq, 
                        tol,
                        verbose)
    # PingPong
    overlap = pingpong(P0, 
                       Pstar, Qstar, 
                       X, Y, A_eq, b_eq, 
                       pingpong_plot, 
                       verbose
                      )
    # Report results
    print("Overlap=" + str(overlap) 
          + " for "+str(optim_function.__name__)
          +" with n="+str(n)
          +", d="+str(d)
          +", sig="+str(sigma) 
          +", P0_steps="+("0" if P0_steps<10 else "")+str(P0_steps)
          +", stepsize="+str(stepsize)
         )
    

In [5]:
for sigma in [0.05, 0.1, 0.2]:
    for d in [10, 20, 50]:
        print("----- d="+str(d)+ ", sigma="+str(sigma)+" -----")
        for P0_steps in [1, 20, 100, 1000]:
            n = 100
            stepsize = 1/100
            experiment(frank_wolfe, n, d, sigma, P0_steps, stepsize, verbose=0)

----- d=10, sigma=0.05 -----


ValueError: Invalid input for linprog: unable to interpret bounds with this dimension tuple: (100000000, 2).

In [ ]:
def Birkhoff_projection(M):
    # Projects matrix onto the closest doubly stochastic matrix
    ...

def projected_gradient_descent(D_init, X, Y, step_max, stepsize, A_eq, b_eq, tol=0.1, verbose=1):
    if verbose>0: print("Running Projected Gradient Descent with "+str(step_max)+ " iterations and step size "+str(stepsize))
    D = D_init.copy()
    if step_max == 0: return D
    n = len(D)
    if verbose>0: print("Loss value before iterations = "+str(f(D, X, Y)))
    step = 1
    while True:
        if verbose>1:print("Projected GD step no. "+str(step))
        G = grad_f(D, X, Y)
        D = D - stepsize*G
        
#         D = Birkhoff_projection(D, A_eq, b_eq)
        
        if step == step_max:
                if verbose>0: print("Reached max number of iterations.")
                break
        step += 1
    if verbose>0: print("Loss value after iterations = "+str(f(D, X, Y)))
    return D

In [ ]:
n=100
d=15
sigma=0.4
steps=50
stepsize=1/50

evaluate_convergence(frank_wolfe, n, d, sigma, steps, stepsize)
evaluate_convergence(projected_gradient_descent, n, d, sigma, steps, stepsize)

# Augmentation via belief propagation
This seems already better now. Let's try to augment the solution since we're super far from the true one.

Other Todos: 
- Reimplement the stuff from Berthet paper and see how we're doing
- Do the descent from Berthet and see how it compares

In [ ]:
tronc_lines = np.zeros((n, n))
for i in range(n):
    j = np.argmax(D[i, :])
    tronc_lines[i,j] = 1
    
plt.imshow(tronc_lines, cmap='hot')
plt.title("Keeping only maximising index of each line in D")
plt.show()

print("Loss trajectory when initialising with max line element")
P, Q = augment_PandQ(X, Y, tronc_lines, A_eq, b_eq)
print(empirical_loss(X, Y, P, Q))
for i in range(4):
    P, Q = augment_PandQ(X, Y, P, A_eq, b_eq)
    print(loss(X, Y, P, Q))
print("Loss at (P*, Q*) equals: " + str(empirical_loss(X, Y, Pstar, Qstar)))

# Archive, code not needed anymore

In [ ]:
print("Histograms for the doubly stochastic matrix")


vecD = D.flatten()
plt.figure(figsize=(8,5))
plt.hist(vecD, bins=50, rwidth = 0.7)
plt.title("histogram values in vecD, LOGSCALE")
plt.yscale("log")
plt.show()


n_plots = 5

plt.figure(figsize=(12, 18))
for i in range(n_plots):
    plt.subplot(n_plots, 2, 2*i+1)
    plt.hist(vecD[i*n:(i+1)*n], bins=50, rwidth = 0.7)
    plt.title("Histogram of LINE number " +str(i+1)+" of D")
    plt.yscale("log")
    
    plt.subplot(n_plots, 2, 2*i+2)
    plt.hist(vecD[i::n], bins=50, rwidth = 0.7, color="red")
    plt.title("Histogram of COLUMN number " +str(i+1)+" of D")
    plt.yscale("log")

plt.show()

In [ ]:
P0 = perm_projection(D, A_eq, b_eq)
P, Q = augment_PandQ(X, Y, P0, A_eq, b_eq)
print("Loss trajectory for initialisation at P0 = Proj_Perm(D)")
print(empirical_loss(X, Y, P, Q))
for i in range(6):
    P, Q = augment_PandQ(X, Y, P, A_eq, b_eq)
    print(empirical_loss(X, Y, P, Q))
print("Loss at (P*, Q*) equals: " + str(empirical_loss(X, Y, Pstar, Qstar)))

In [ ]:
# Some other matrices to check out the alternative optim algo.
pirand = np.random.permutation(n)
Prand = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if pirand[i] == j:
            Prand[i,j] = 1
            
PprojD = perm_projection(D, A_eq, b_eq)

D_tronc_lines = np.zeros((n, n))
for i in range(n):
    j = np.argmax(D[i, :])
    D_tronc_lines[i,j] = 1
print("Defined Prand, PprojD, D_tronc_lines")
pingpong(Prand, X, Y, A_eq, b_eq)